In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Loading data

In [ ]:
df = pd.read_csv('ex8data1_X.csv', names=['Latency', 'Throughput'])

df_x_val = pd.read_csv('ex8data1_Xval.csv', names=['Latency', 'Throughput'])
df_y_val = pd.read_csv('ex8data1_yval.csv', names=['IsOutlier'])

In [ ]:
df.plot(kind='scatter',x='Latency',y='Throughput')

In [ ]:
ds = tf.contrib.distributions
sess = tf.Session()

Generate normal distribution

In [ ]:
mvn = ds.MultivariateNormalDiag(
    loc=[df.mean().Latency, df.mean().Throughput],
    scale_diag=[df.std().Latency, df.std().Throughput])

In [ ]:
t_mean = mvn.mean()
t_std = mvn.stddev()
with sess.as_default():
    print sess.run([t_mean, t_std])

Calculate each data probability

In [ ]:
tv = tf.to_float(df.values, name='ToFloat')
t_prob = mvn.prob(tv)

tv_x_val = tf.to_float(df_x_val.values, name='ToFloat')
t_prob_val = mvn.prob(tv_x_val)

with sess.as_default():
    sess.run([t_prob,t_prob_val])

Convert to np array

In [ ]:
t_prob_val = tf.Print(t_prob_val, [t_prob_val])
tv_y_val = tf.to_float(df_y_val.values, name='ToFloat')
t_prob = tf.Print(t_prob, [t_prob])

with sess.as_default():
    prob = sess.run(t_prob)
    prob_val = sess.run(t_prob_val)
    y_val = sess.run(tv_y_val)
y_val = y_val.flatten()

In [ ]:
x_max = np.max(prob_val)
x_min = np.min(prob_val)
x_max, x_min

Pick the best epsilon

In [ ]:
best_epsilon = 0
best_f1 = 0
f1 = 0

step = (x_max - x_min) / 1000
for epsilon in np.arange(x_min, x_max, step):
    preds = prob_val < epsilon
    
    tp = np.sum(np.logical_and(preds == 1, y_val == 1)).astype(float)
    fp = np.sum(np.logical_and(preds == 1, y_val == 0)).astype(float)
    fn = np.sum(np.logical_and(preds == 0, y_val == 1)).astype(float)
        
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = (2 * precision * recall) / (precision + recall)

    if f1 > best_f1:
        best_f1 = f1
        best_epsilon = epsilon
best_epsilon, best_f1, f1

Determime whether it is outlier or not

In [ ]:
outliers = np.where(prob_val < best_epsilon)

Add outlier column at dataframe

In [ ]:
df['IsOutlier'] = False
for outlier in outliers[0]:
    df.iloc[outlier, df.columns.get_loc('IsOutlier')] = True
df.loc[df['IsOutlier'] == True]

In [ ]:
ax = df.loc[df['IsOutlier'] == True].plot(
    kind='scatter',x='Latency',y='Throughput',color='Red',label='Outlier')
df.loc[df['IsOutlier'] == False].plot(
    kind='scatter',x='Latency',y='Throughput',color='Green',label='Normal',ax=ax)